In [1]:
!pip install antlr4-python3-runtime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 3.7 MB/s eta 0:00:00


In [2]:
!curl -O https://www.antlr.org/download/antlr-4.13.2-complete.jar

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2089k  100 2089k    0     0  3736k      0 --:--:-- --:--:-- --:--:-- 3731k


In [3]:
%%file parcial.g4
grammar parcial;
program
    : statement+ ;

statement
    : decVar SEMI                                  # printDecVar
    | expr SEMI                                    # printExpr
    | functionDec                                  # printFunction
    | ifStatement                                  # ifStatemen
    ;

decVar
    : VAR_CHAR varDecl (COMMA varDecl)*           # decChar
    | VAR_INT varDecl (COMMA varDecl)*            # decInt
    | VAR_FLOAT varDecl (COMMA varDecl)*          # decFloat
    | VAR_DOUBLE varDecl (COMMA varDecl)*         # decDouble
    ;

varDecl
    : ID (CL expr? CR)? (ASSIGN literal)? ; // Declaración clara de una variable

ifStatement
    : IF LPAREN expr RPAREN LLLAVE body RLLAVE (ELSE LLLAVE body RLLAVE)?
    ;

expr
    : expr PLUS expr                             # sumsub
    | expr MINUS expr                            # sumsub
    | expr MUL expr                              # muldiv
    | expr DIV expr                              # muldiv
    | expr MOD expr                              # modulo
    | expr POT expr                              # pow
    | expr LT expr                               # menorque
    | expr GT expr                               # mayorque
    | expr NEQ expr                              # noequal
    | CL expr? CR                                # corchete
    | LPAREN expr RPAREN                         # parentesis
    | literal                                    # literalexpr
    | ID                                         # variable
    | INPUT expr                                 # input
    | OUTPUT expr                                # output
    ;

functionDec
    : FUNC literal LPAREN RPAREN LLLAVE body RET expr RLLAVE ; // Declaración de función

paramList
    : VAR_INT ID (COMMA VAR_INT ID)* ; // Lista de parámetros para funciones

body
    : statement* ; // Cuerpo de la función

literal
    : NUM                                       # numberLiteral
    | STRING                                    # stringLiteral
    ;

// Lexer rules


WS       : [ \t\r\n]+ -> skip ;           // Ignorar espacios en blanco
FUNC     : 'function';                    // Funcion
PLUS     : 'sum' | '+';                   // Suma
MINUS    : 'res' | '-';                   // Resta
MUL      : 'mul' | '*';                   // Multiplicación
DIV      : 'div' | '/';                   // División
MOD      : '%';                           // Modular
POT      : 'pot' | '^';                   // Potencia
ASSIGN   : '=';                           // Asignación
INC      : '++';                          // Incremento
LLLAVE   : '{';                           // Llave izquierda
RLLAVE   : '}';                           // Llave derecha
CL       : '[';
CR       : ']';
LPAREN   : '(';                           // Paréntesis izquierdo
RPAREN   : ')';                           // Paréntesis derecho
LT       : '<';                           // Menor que
GT       : '>';                           // Mayor que
NEQ      : '<>';                          // Diferente
IF       : 'if';                          // Condicional
ELSE     : 'else';                        // Alternativa
FOR      : 'for';                         // Bucle
RET      : 'ret';                         // Retorno
VAR_INT  : ':n';                          // Variable entera
VAR_FLOAT: ':nF';                         // Variable float
VAR_DOUBLE: ':nD';                        // Variable double
VAR_CHAR : ':Ch';                         // Variable char
VAR_STR  : ':S';                          // String
STRUCT   : 'estruct';                     // Definición de estructura
OUTPUT   : '->';                          // Equivalente a cout
INPUT    : '<-';                          // Equivalente a cin
COMMA    : ',';                           // Separador
SEMI     : ';';                           // Terminador de declaraciones
ID       : [a-zA-Z][a-zA-Z0-9_]* ;        // Identificadores
NUM      : [0-9]+ ('.'[0-9]+)? ;          // Números
STRING   : '"' .*? '"' ;                  // Cadenas de texto


Writing parcial.g4


In [4]:
%%bash
java -cp antlr-4.13.2-complete.jar:. org.antlr.v4.Tool -Dlanguage=Python3 parcial.g4

In [5]:
%%file ejemplo.parcial

:n x = 10;
:n z = 20;
:n r = 5;
:n a = 2;
r + r;
r * x;
r / x;
:n myVar = 3;
:n result;

if (x < z) {
    -> "x es menor que z";
} else {
    -> "x no es menor que z";
}


Writing ejemplo.parcial


In [6]:
from typing_extensions import runtime
import antlr4
from parcialLexer import parcialLexer
from parcialParser import parcialParser

from antlr4.CommonTokenStream import CommonTokenStream
from antlr4 import FileStream
from antlr4.error.ErrorListener import ErrorListener


In [7]:
input = FileStream("ejemplo.parcial")
lexer = parcialLexer(input)
stream = CommonTokenStream(lexer)
parser = parcialParser(stream)
tree = parser.program()
print(tree.toStringTree(recog=parser))

(program (statement (decVar :n (varDecl x = (literal 10))) ;) (statement (decVar :n (varDecl z = (literal 20))) ;) (statement (decVar :n (varDecl r = (literal 5))) ;) (statement (decVar :n (varDecl a = (literal 2))) ;) (statement (expr (expr r) + (expr r)) ;) (statement (expr (expr r) * (expr x)) ;) (statement (expr (expr r) / (expr x)) ;) (statement (decVar :n (varDecl myVar = (literal 3))) ;) (statement (decVar :n (varDecl result)) ;) (statement (ifStatement if ( (expr (expr x) < (expr z)) ) { (body (statement (expr -> (expr (literal "x es menor que z"))) ;)) } else { (body (statement (expr -> (expr (literal "x no es menor que z"))) ;)) })))


In [8]:
from parcialListener import parcialListener

class parcialDriver(parcialListener):
    def enterProgram(self, ctx):
        print("Entering program...")

    def exitProgram(self, ctx):
        print("Exiting program...")

    def enterPrintDecVar(self, ctx):
        print("Entering a variable declaration statement.")

    def exitPrintDecVar(self, ctx):
        print("Exiting a variable declaration statement.")

    def enterPrintExpr(self, ctx):
        print("Entering an expression statement.")

    def exitPrintExpr(self, ctx):
        print("Exiting an expression statement.")

    def enterPrintFunction(self, ctx):
        print("Entering a function declaration.")

    def exitPrintFunction(self, ctx):
        print("Exiting a function declaration.")

    def enterDecChar(self, ctx):
        print("Declaring a character variable.")

    def exitDecChar(self, ctx):
        print("Character variable declaration completed.")

    def enterDecInt(self, ctx):
        print("Declaring an integer variable.")

    def exitDecInt(self, ctx):
        print("Integer variable declaration completed.")

    def enterDecFloat(self, ctx):
        print("Declaring a float variable.")

    def exitDecFloat(self, ctx):
        print("Float variable declaration completed.")

    def enterDecDouble(self, ctx):
        print("Declaring a double variable.")

    def exitDecDouble(self, ctx):
        print("Double variable declaration completed.")

    def enterLiteralexpr(self, ctx):
        print("Processing a literal expression.")

    def exitLiteralexpr(self, ctx):
        print("Completed processing a literal expression.")

    def enterSumsub(self, ctx):
        print("Processing a sum or subtraction.")

    def exitSumsub(self, ctx):
        print("Completed processing a sum or subtraction.")

    def enterMuldiv(self, ctx):
        print("Processing multiplication or division.")

    def exitMuldiv(self, ctx):
        print("Completed processing multiplication or division.")

    def enterOutput(self, ctx):
        print("Processing an output operation.")

    def exitOutput(self, ctx):
        print("Completed processing an output operation.")

    def enterInput(self, ctx):
        print("Processing an input operation.")

    def exitInput(self, ctx):
        print("Completed processing an input operation.")

    def enterFunctionDec(self, ctx):
        print(f"Entering a function definition: {ctx.getText()}")

    def exitFunctionDec(self, ctx):
        print("Exiting the function definition.")

    def enterPow(self, ctx):
        print("Processing a power operation.")

    def exitPow(self, ctx):
        print("Completed processing a power operation.")

    def enterModulo(self, ctx):
        print("Processing a modulo operation.")

    def exitModulo(self, ctx):
        print("Completed processing a modulo operation.")

    def enterIfStatement(self, ctx):
        print("Entering an if statement.")

    def exitIfStatement(self, ctx):
        print("Exiting an if statement.")

    def enterVariable(self, ctx):
        print(f"Encountered a variable: {ctx.getText()}")

    def exitVariable(self, ctx):
        print(f"Finished processing variable: {ctx.getText()}")



In [9]:
from antlr4 import ParseTreeWalker
input = FileStream("ejemplo.parcial")
lexer = parcialLexer
tokens = CommonTokenStream(lexer(input))
parser = parcialParser(tokens)
tree = parser.program()
walker = ParseTreeWalker()
walker.walk(parcialDriver(), tree)

Entering program...
Entering a variable declaration statement.
Declaring an integer variable.
Integer variable declaration completed.
Exiting a variable declaration statement.
Entering a variable declaration statement.
Declaring an integer variable.
Integer variable declaration completed.
Exiting a variable declaration statement.
Entering a variable declaration statement.
Declaring an integer variable.
Integer variable declaration completed.
Exiting a variable declaration statement.
Entering a variable declaration statement.
Declaring an integer variable.
Integer variable declaration completed.
Exiting a variable declaration statement.
Entering an expression statement.
Processing a sum or subtraction.
Encountered a variable: r
Finished processing variable: r
Encountered a variable: r
Finished processing variable: r
Completed processing a sum or subtraction.
Exiting an expression statement.
Entering an expression statement.
Processing multiplication or division.
Encountered a variable: 

In [10]:
class parcialDriver(parcialListener):
    def __init__(self):
        self.llvm_ir = ""
        self.variables = {}  # Diccionario para almacenar variables y referencias
        self.tmp_counter = 0  # Contador para nombres temporales
        self.label_counter = 0  # Contador para etiquetas

    def get_tmp(self):
        tmp_name = f"%tmp{self.tmp_counter}"
        self.tmp_counter += 1
        return tmp_name

    def get_label(self):
        label_name = f"label{self.label_counter}"
        self.label_counter += 1
        return label_name

    def enterProgram(self, ctx):
        self.llvm_ir += """
declare i32 @printf(i8*, ...)
@format_int = private unnamed_addr constant [4 x i8] c"%d\\0A\\00"
@.str_true = private unnamed_addr constant [18 x i8] c"x es menor que z\\0A\\00", align 1
@.str_false = private unnamed_addr constant [21 x i8] c"x no es menor que z\\0A\\00", align 1

define i32 @main() {
"""

    def exitProgram(self, ctx):
        self.llvm_ir += "  ret i32 0\n}\n"
        print(self.llvm_ir)

###DECLARACIONES
    def exitDecInt(self, ctx):
        second_child_text = ctx.getChild(1).getText()
        if '=' in second_child_text:
            var_name, value = second_child_text.split('=')
            var_name, value = var_name.strip(), value.strip()
            self.llvm_ir += f"  %{var_name} = alloca i32\n"
            self.llvm_ir += f"  store i32 {value}, i32* %{var_name}\n"
            self.variables[var_name] = f"%{var_name}"
        else:
            var_name = second_child_text.strip()
            self.llvm_ir += f"  %{var_name} = alloca i32\n"
            self.variables[var_name] = f"%{var_name}"

    def resolve_expr(self, expr_ctx):
        if isinstance(expr_ctx, parcialParser.LiteralexprContext):
            literal = expr_ctx.literal()
            if isinstance(literal, parcialParser.NumberLiteralContext):  # Literal numérico
                return literal.NUM().getText()
            elif isinstance(literal, parcialParser.StringLiteralContext):  # Literal de cadena
                return literal.STRING().getText()
        elif isinstance(expr_ctx, parcialParser.VariableContext):
            var_name = expr_ctx.ID().getText()
            return f"load i32, i32* %{var_name}"
###OPERACIONES
    def exitSumsub(self, ctx):
        left = self.resolve_expr(ctx.expr(0))
        right = self.resolve_expr(ctx.expr(1))
        op = ctx.getChild(1).getText()
        result_tmp = self.get_tmp()

        left_val = self.get_tmp()
        self.llvm_ir += f"  {left_val} = {left}\n"

        right_val = self.get_tmp()
        self.llvm_ir += f"  {right_val} = {right}\n"

        if op == '+':
            self.llvm_ir += f"  {result_tmp} = add i32 {left_val}, {right_val}\n"
        elif op == '-':
            self.llvm_ir += f"  {result_tmp} = sub i32 {left_val}, {right_val}\n"

        self.variables["last_result"] = result_tmp

    def exitMuldiv(self, ctx):
        left = self.resolve_expr(ctx.expr(0))
        right = self.resolve_expr(ctx.expr(1))
        op = ctx.getChild(1).getText()
        result_tmp = self.get_tmp()

        left_val = self.get_tmp()
        self.llvm_ir += f"  {left_val} = {left}\n"

        right_val = self.get_tmp()
        self.llvm_ir += f"  {right_val} = {right}\n"

        if op == '*':
            self.llvm_ir += f"  {result_tmp} = mul i32 {left_val}, {right_val}\n"
        elif op == '/':
            self.llvm_ir += f"  {result_tmp} = sdiv i32 {left_val}, {right_val}\n"

        self.variables["last_result"] = result_tmp

    def exitModulo(self, ctx):
        left = self.resolve_expr(ctx.expr(0))
        right = self.resolve_expr(ctx.expr(1))
        result_tmp = self.get_tmp()

        left_val = self.get_tmp()
        self.llvm_ir += f"  {left_val} = {left}\n"

        right_val = self.get_tmp()
        self.llvm_ir += f"  {right_val} = {right}\n"

        self.llvm_ir += f"  {result_tmp} = srem i32 {left_val}, {right_val}\n"

        self.variables["last_result"] = result_tmp

    def exitPow(self, ctx):
        left = self.resolve_expr(ctx.expr(0))
        right = self.resolve_expr(ctx.expr(1))
        result_tmp = self.get_tmp()

        left_val = self.get_tmp()
        self.llvm_ir += f"  {left_val} = {left}\n"

        right_val = self.get_tmp()
        self.llvm_ir += f"  {right_val} = {right}\n"

        self.llvm_ir += f"  {result_tmp} = call float @llvm.pow.f32(float {left_val}, float {right_val})\n"

        self.variables["last_result"] = result_tmp

    def exitMenorque(self, ctx):
        left = self.resolve_expr(ctx.expr(0))
        right = self.resolve_expr(ctx.expr(1))

        left_tmp = self.get_tmp()
        self.llvm_ir += f"  {left_tmp} = {left}\n"

        right_tmp = self.get_tmp()
        self.llvm_ir += f"  {right_tmp} = {right}\n"

        result_tmp = self.get_tmp()
        self.llvm_ir += f"  {result_tmp} = icmp slt i32 {left_tmp}, {right_tmp}\n"

        self.variables["last_result"] = result_tmp

    def exitMayorque(self, ctx):
        left = self.resolve_expr(ctx.expr(0))
        right = self.resolve_expr(ctx.expr(1))

        left_tmp = self.get_tmp()
        self.llvm_ir += f"  {left_tmp} = {left}\n"

        right_tmp = self.get_tmp()
        self.llvm_ir += f"  {right_tmp} = {right}\n"

        result_tmp = self.get_tmp()
        self.llvm_ir += f"  {result_tmp} = icmp sgt i32 {left_tmp}, {right_tmp}\n"

        self.variables["last_result"] = result_tmp


    def exitNoequal(self, ctx):
        left = self.resolve_expr(ctx.expr(0))
        right = self.resolve_expr(ctx.expr(1))
        result_tmp = self.get_tmp()
        self.llvm_ir += f"  {result_tmp} = icmp ne i32 {left}, {right}\n"
        self.variables["last_result"] = result_tmp

###IF STATEMENT
    def exitIfStatement(self, ctx):
        condition = self.resolve_expr(ctx.expr())
        true_label = self.get_label()
        false_label = self.get_label()
        end_label = self.get_label()

        self.llvm_ir += f"  br i1 %tmp11, label %{true_label}, label %{false_label}\n"

        self.llvm_ir += f"{true_label}:\n"
        self.llvm_ir += f"  call i32 (i8*, ...) @printf(i8* getelementptr ([18 x i8], [18 x i8]* @.str_true, i32 0, i32 0))\n"
        self.llvm_ir += f"  br label %{end_label}\n"

        self.llvm_ir += f"{false_label}:\n"
        self.llvm_ir += f"  call i32 (i8*, ...) @printf(i8* getelementptr ([21 x i8], [21 x i8]* @.str_false, i32 0, i32 0))\n"
        self.llvm_ir += f"  br label %{end_label}\n"

        self.llvm_ir += f"{end_label}:\n"

    def save_ir_to_file(self, filename):
        with open(filename, 'w') as file:
            file.write(self.llvm_ir)

# Procesar el archivo de entrada
input_file = "ejemplo.parcial"
output_file = "output.ll"

input_stream = FileStream(input_file)
lexer = parcialLexer(input_stream)
tokens = CommonTokenStream(lexer)
parser = parcialParser(tokens)
tree = parser.program()

driver = parcialDriver()
walker = ParseTreeWalker()
walker.walk(driver, tree)

# Guardar el resultado
driver.save_ir_to_file(output_file)
print(f"Código LLVM IR guardado en: {output_file}")



declare i32 @printf(i8*, ...)
@format_int = private unnamed_addr constant [4 x i8] c"%d\0A\00"
@.str_true = private unnamed_addr constant [18 x i8] c"x es menor que z\0A\00", align 1
@.str_false = private unnamed_addr constant [21 x i8] c"x no es menor que z\0A\00", align 1

define i32 @main() {
  %x = alloca i32
  store i32 10, i32* %x
  %z = alloca i32
  store i32 20, i32* %z
  %r = alloca i32
  store i32 5, i32* %r
  %a = alloca i32
  store i32 2, i32* %a
  %tmp1 = load i32, i32* %r
  %tmp2 = load i32, i32* %r
  %tmp0 = add i32 %tmp1, %tmp2
  %tmp4 = load i32, i32* %r
  %tmp5 = load i32, i32* %x
  %tmp3 = mul i32 %tmp4, %tmp5
  %tmp7 = load i32, i32* %r
  %tmp8 = load i32, i32* %x
  %tmp6 = sdiv i32 %tmp7, %tmp8
  %myVar = alloca i32
  store i32 3, i32* %myVar
  %result = alloca i32
  %tmp9 = load i32, i32* %x
  %tmp10 = load i32, i32* %z
  %tmp11 = icmp slt i32 %tmp9, %tmp10
  br i1 %tmp11, label %label0, label %label1
label0:
  call i32 (i8*, ...) @printf(i8* getelementptr ([18 x